Read pulsar data taken from the CSIRO DAP at  
https://data.csiro.au/dap/search?q=pulsars  
and interpret according to psrfits format described at  
http://www.atnf.csiro.au/people/pulsar/index.html?n=Main.Psrfits  
(detailed view at http://www.atnf.csiro.au/people/pulsar/index.html?n=PsrfitsDocumentation.Txt)  

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import scipy.signal as ss
import astropy
from astropy.io import fits
import matplotlib.pyplot as plt

In [ ]:
cd ~/pulsarvoices/data

In [ ]:
ls

In [ ]:
hdulist = fits.open('p130712_234328.rf')

In [ ]:
hdulist.info()

In [ ]:
# Object name
hdulist['PSRPARAM'].data[0][0]

In [ ]:
hdulist['PSRPARAM'].data

In [ ]:
type(hdulist['SUBINT'].data)

In [ ]:
from astropy.io.fits import getdata

In [ ]:
getdata('p130712_234328.rf', 'SUBINT', 0)

In [ ]:
freq_bins = hdulist['SUBINT'].data[0][-5]

In [ ]:
hdulist['SUBINT'].data[0][-4].shape

In [ ]:
plt.plot(hdulist['SUBINT'].data[0][-5])

In [ ]:
data1 = hdulist['SUBINT'].data[0][-1]
data1.shape

In [ ]:
# From http://stackoverflow.com/questions/2459295/invertible-stft-and-istft-in-python
import scipy, pylab

def stft(x, fs, framesz, hop):
    framesamp = int(framesz*fs)
    hopsamp = int(hop*fs)
    w = scipy.hanning(framesamp)
    X = scipy.array([scipy.fft(w*x[i:i+framesamp]) 
                     for i in range(0, len(x)-framesamp, hopsamp)])
    return X

def istft(X, fs, T, hop):
    x = scipy.zeros(T*fs)
    framesamp = X.shape[1]
    hopsamp = int(hop*fs)
    for n,i in enumerate(range(0, len(x)-framesamp, hopsamp)):
        x[i:i+framesamp] += scipy.real(scipy.ifft(X[n]))
    return x

def istft2(X, overlap=4):   
    fftsize=(X.shape[1]-1)*2
    hop = fftsize / overlap
    w = scipy.hanning(fftsize+1)[:-1]
    x = scipy.zeros(X.shape[0]*hop)
    wsum = scipy.zeros(X.shape[0]*hop) 
    for n,i in enumerate(range(0, len(x)-fftsize, hop)): 
        x[i:i+fftsize] += scipy.real(np.fft.irfft(X[n])) * w   # overlap-add
        wsum[i:i+fftsize] += w ** 2.
    pos = wsum != 0
    x[pos] /= wsum[pos]
    return x

In [ ]:
polar1 = data1[0]
chirp = polar1[100:450]

In [ ]:
plt.imshow(chirp, origin='lower')
plt.colorbar()

In [ ]:
fs = 16000
T = 2
hop = 200./fs
# isf = istft(data1[0].T, fs, T, hop)
isf = istft2(chirp, overlap=300)

In [ ]:
from scipy.io import wavfile

def write_wavfile(data, fs, filename='pulsarsound.wav'):
    d = np.int16(data * 0.9 / np.max(np.abs(data)) * 32767)
    wavfile.write(filename, int(fs), d)

def note(sig, time, fname='pulsarsound.wav'):
    rate = int(float(sig.size) / time)
    print rate
    wavfile.write(fname, rate, sig)

write_wavfile(isf, 1000)
#note(isf, 2)

In [ ]:
plt.plot(isf)
plt.ylim(-1e3, 1e3)

In [ ]:
pwd

Docs for
SUBINTEGRATION DATA BINARY TABLE EXTENSION

Table containing the observed power spectra, that is spectra after detection or multiplication. Two modes of observation are catered for:

fold mode: where the data are synchronously folded at the apparent period of a pulsar using a Tempo polyco file or a Tempo2 predictor file and samples are binned in pulse phase
search mode: where streamed multichannel data are recorded in successive samples.
In both modes, the data can have a single polarisation (normally the sum of two orthogonal polarisations), two orthogonal polarisations or all four polarisation spectra.

In fold mode, data are summed over a sub-integration time and successive sub-integrations are stored in successive rows of the BINTABLE. In search mode, data are blocked in groups of NSBLK samples and stored in successive rows of the BINTABLE. To avoid excessive overheads, NSBLK is typically 4096.

Fold-mode data are stored as 16-bit signed integers with elements of the data array in bin, channel and polarisation order with the pulse profile bins in contiguous locations. Before conversion to integers, the mean channel power (averaged over bins and polarisations) during the sub-integration is subtracted from the channel data and the residual is scaled so that the values in the DATA array cover the whole available range (-32768 to 32767). The original observed powers are reconstructed using:

Real value = DATA value * DAT_SCL + DAT_OFFS.
Search-mode data may be stored as 1-bit, 2-bit, 4-bit or 8-bit signed or unsigned integers and are written as a byte array. Data digitised with less than 8 bits are packed with earlier samples in higher-order bits of the byte (i.e., "big-endian"). Elements of the data array are in channel, polarisation and sample order with the spectral channels in contiguous locations.

Prior to few-bit digitisation, search-mode sample spectra are generally normalised and given zero mean by forming (S-R)/R, where S is the observed spectrum and R is an estimate of the bandpass or reference spectrum. This effectively does a bandpass calibration and gives an approximately constant rms deviation across the spectrum in order to optimise the few-bit digitisation. These data are normally analysed directly without application of the scale factors and offsets. If required, the reference spectrum may be reconstructed from the DAT_OFFS and DAT_SCL fields of the table. However note that, if channel running means are used to form the reference spectrum, the recorded values are sampled at sub-integration intervals and do do not necessarily represent the exact values used to form the recorded spectra.

When unsigned integers are used to record the truncated data, a zero offset (ZERO_OFFS) is added to the digitised value. Normally ZERO_OFFS = 2^(NBIT - 1) - 0.5, but for total-intensity multi-bit data, a smaller value may be used to give more headroom. The original observed values are reconstructed using:

Real value = (DATA value - ZERO_OFFS) * DAT_SCL + DAT_OFFS.
The output data for long search-mode observations may be split in time or frequency and recorded in separate files to keep file sizes at manageable values. It is assumed that the data sampling is continuous across the split files.

XTENSION= BINTABLE / ***** Subintegration data ***** 
BITPIX = 8 / N/A 
NAXIS = 2 / 2-dimensional binary table 
NAXIS1 = * / width of table in bytes 
NAXIS2 = * / Number of rows in table (NSUBINT) 
PCOUNT = 0 / size of special data area 
GCOUNT = 1 / one data group (required keyword) 
TFIELDS = 18 / Number of fields per row

Standard FITS Binary Table header
INT_TYPE= ' ' / Time axis (TIME, BINPHSPERI, BINLNGASC, etc)

Normally data are recorded with a uniform time axis, i.e., sub-integrations are spaced by equal time intervals. However, in post-processing it is possible to make the nominal time axis non-linear, to represent for example, binary orbital phase or longitude. This keyword describes the form of the nominal time axis.
INT_UNIT= ' ' / Unit of time axis (SEC, PHS (0-1), DEG)

Units of the nominal time axis.
SCALE = ' ' / Intensity units (FluxDen/RefFlux/Jansky)

Units of the data.
POL_TYPE= ' ' / Polarisation identifier (e.g., AABBCRCI, AA+BB)

A string describing the type of polarisation data, e.g., for NPOL=1 total-intensity data (summed orthogonal polarisations), "AA+BB" may be used, where AA represents the auto-spectrum of the A channel and BB represents the auto-spectrum of the B channel. For NPOL=2, "AABB" may be used. For full polarisation data, "AABBCRCI" may be used where CR represents the real part of the A* B cross-spectrum and CI represents the imaginary part. For calibrated Stokes parameters, "IQUV" is normally used.
NPOL = * / Nr of polarisations

The number of polarisation products contained in the DATA table.
TBIN = * / [s] Time per bin or sample

For fold-mode data, the bin width in seconds; for search-mode data, the sampling interval in seconds.
NBIN = * / Nr of bins (PSR/CAL mode; else 1)

The number of bins in the fold-mode profile. For search-mode data, this parameter is irrelevant and normally set to 1.
NBIN_PRD= * / Nr of bins/pulse period (for gated data)

For observations where only part of the pulse profile is recorded (gated data), NBIN_PRD is the number of bins that the full pulse period would occupy. For example, if 256 bins are recorded across one quarter of the pulse period, NBIN = 256 and NBIN_PRD = 1024.
Alternatively, if more than one pulse period is contained within the recorded pulse profile, then NBIN_PRD is a sub-multiple of NBIN.

PHS_OFFS= * / Phase offset of bin 0 for gated data

For gated data, the value of pulse phase (in the range 0 - 1.0) corresponding to the leading edge of the first (index zero) bin in the folded profile.
NBITS = * / Nr of bits/datum (SEARCH mode data, else 1)

For search-mode data, the number of bits per sample datum. Ignored for fold-mode data.
ZERO_OFF= * / Zero offset for SEARCH-mode data

For data recorded as unsigned integers, the offset of the nominal data zero in digitiser units (actual data value = unsigned integer value - ZERO_OFF).
SIGNINT = * / 1 for signed ints in SEARCH-mode data, else 0

A flag to indicate that the data values are signed integers rather than unsigned integers.
NSUBOFFS= * / Subint offset (Contiguous SEARCH-mode files)

For files split in time, NSUBOFFS is the accumulated sub-integration count at the start of this file (such that true index value = file index value + NSUBOFFS).
NCHAN = * / Number of channels/sub-bands in this file

Number of frequency channels for each polarisation
CHAN_BW = * / [MHz] Channel/sub-band width

Channel bandwidth in MHz. Note that CHAN_BW < 0 implies decreasing frequency with increasing channel number.
DM = * / [cm-3 pc] DM for post-detection dedisperion

Dispersion measure used for post-detection dedispersion.
RM = * / [rad m-2] RM for post-detection deFaraday

Rotation measure used for post-detection correction of Faraday rotation
NCHNOFFS= * / Channel/sub-band offset for split files

For files split in frequency, NCHNOFFS is the true channel index of the first channel in this file (true index value = file index value + NCHNOFFS).
NSBLK = * / Samples/row (SEARCH mode, else 1)

For search-mode data, the number of samples in one table row or sub-integration.
NSTOT = * / Total number of samples (SEARCH mode, else 1)

Total number of valid samples in a search-mode file. This is useful since the last row may not be completely filled. Note, for split files, NSTOT should be the number of samples in that file, i.e., not accumulated over the whole observation.
EXTNAME = SUBINT / name of this binary table extension

TTYPE# = INDEXVAL / Optionally used if INT_TYPE != TIME 
TFORM# = 1D / Double

If INT_TYPE is not "TIME", this column gives the value of the time-like coordinate at the sub-integration centre, expressed in appropriate units (e.g., degrees for longitude).
TTYPE# = TSUBINT / Length of subintegration 
TFORM# = 1D / Double 
TUNIT# = s / Units of field

Duration of sub-integration (or row for search-mode data)
TTYPE# = OFFS_SUB / Offset from Start of subint centre 
TFORM# = 1D / Double 
TUNIT# = s / Units of field

Time since the observation start at the centre of each sub-integration (or row).
TTYPE# = LST_SUB / LST at subint centre 
TFORM# = 1D / Double 
TUNIT# = s / Units of field

Approximate local sidereal time at the sub-integration centre
TTYPE# = RA_SUB / RA (J2000) at subint centre 
TFORM# = 1D / Double 
TUNIT# = deg / Units of field

Pointing J2000 Right Ascension at the time of the sub-integration centre. For scanning observations this may change with time.
TTYPE# = DEC_SUB / Dec (J2000) at subint centre 
TFORM# = 1D / Double 
TUNIT# = deg / Units of field

Pointing J2000 Declination at the time of the sub-integration centre. For scanning observations this may change with time.

In [ ]:
from astropy.coordinates import SkyCoord, spherical_to_cartesian

In [ ]:
# Get coords of pulsar (ra, dec)
header = hdulist['PSRPARAM'].data
ra = header[1][0].split()[1]
dec = header[2][0].split()[1]
print ra, dec

In [ ]:
# Convert (ra, dec) to (x, y, z)
c = SkyCoord(ra, dec, frame='icrs', unit='deg')
spherical_to_cartesian(1, c.ra.radian, c.dec.radian)